In [1]:
import pandas as pd

from lsh_utils import load_phoible
from lsh_utils import hashes
from lsh_utils import compare

from simphon import Token
from simphon import tokens

We can look at each token in our toy data set along with its phonemic transcription as a sequence of phonemes (between "/"):



In [2]:
# a toy dataset to work with
print(*tokens, sep='\n')

(eng) Alex /æ l ə k s/
(eng) Alexander /æ l ə k z æ n d ɚ/
(eng) Alexi /ə l ɛ k s i/
(eng) Alexis /ə l ɛ k s ɪ s/
(eng) Andrew /æ n d ɹ u/
(eng) Brad /b ɹ æ d/
(eng) Bradley /b ɹ æ d l i/
(eng) Brett /b ɹ ɛ t/
(eng) Carl /k ɑ ɹ l/
(eng) Carlos /k ɑ ɹ l oʊ s/
(eng) Catherine /k æ θ ə ɹ ə n/
(eng) Catherine /k æ θ ə ɹ ɪ n/
(eng) Charles /t̠ʃ ɑ ɹ l z/
(eng) Drew /d ɹ u/
(eng) Jennifer /d̠ʒ ɛ n ə f ɚ/
(eng) Jenny /d̠ʒ ɛ n i/
(eng) John /d̠ʒ ɑ n/
(eng) Johnny /d̠ʒ ɑ n i/
(eng) Jonathan /d̠ʒ ɑ n ə θ ə n/
(eng) Kat /k æ t/
(eng) Kathy /k æ θ i/
(eng) Matt /m æ t/
(eng) Matthew /m æ θ j u/
(eng) Michael /m a ɪ k ə l/
(eng) Mike /m aɪ k/
(eng) Nate /n eɪ t/
(eng) Nathan /n eɪ θ ə n/
(eng) Nathaniel /n ə θ æ n j ə l/
(eng) Nichole /n ɪ k oʊ l/
(eng) Nick /n ɪ k/
(eng) Phil /f ɪ l/
(eng) Philip /f ɪ l ɪ p/
(eng) Ty /t aɪ/
(eng) Tyler /t aɪ l ə ɹ/
(eng) Xander /z æ n d ɚ/
(eng) Zach /z æ k/
(eng) Zachary /z æ k ə ɹ i/
(eng) Zak /z æ k/


We can see the PHOIBLE feature values as a 2-D matrix representation of each token as follows:

In [3]:
from IPython.display import display
pd.options.display.max_columns = None

for token in tokens:
    display(token.as_feature_matrix())

,æ,l,ə,k,s
tone,0,0,0,0,0
stress,-,-,-,-,-
syllabic,+,-,+,-,-
short,-,-,-,-,-
long,-,-,-,-,-
consonantal,-,+,-,+,+
sonorant,+,+,+,-,-
continuant,+,+,+,-,+
delayedRelease,0,0,0,-,+
approximant,+,+,+,-,-


,æ,l,ə,k,z,æ,n,d,ɚ
tone,0,0,0,0,0,0,0,0,0
stress,-,-,-,-,-,-,-,-,-
syllabic,+,-,+,-,-,+,-,-,+
short,-,-,-,-,-,-,-,-,-
long,-,-,-,-,-,-,-,-,-
consonantal,-,+,-,+,+,-,+,+,-
sonorant,+,+,+,-,-,+,+,-,+
continuant,+,+,+,-,+,+,-,-,+
delayedRelease,0,0,0,-,+,0,0,-,0
approximant,+,+,+,-,-,+,-,-,+


,ə,l,ɛ,k,s,i
tone,0,0,0,0,0,0
stress,-,-,-,-,-,-
syllabic,+,-,+,-,-,+
short,-,-,-,-,-,-
long,-,-,-,-,-,-
consonantal,-,+,-,+,+,-
sonorant,+,+,+,-,-,+
continuant,+,+,+,-,+,+
delayedRelease,0,0,0,-,+,0
approximant,+,+,+,-,-,+


,ə,l,ɛ,k,s,ɪ,s
tone,0,0,0,0,0,0,0
stress,-,-,-,-,-,-,-
syllabic,+,-,+,-,-,+,-
short,-,-,-,-,-,-,-
long,-,-,-,-,-,-,-
consonantal,-,+,-,+,+,-,+
sonorant,+,+,+,-,-,+,-
continuant,+,+,+,-,+,+,+
delayedRelease,0,0,0,-,+,0,+
approximant,+,+,+,-,-,+,-


,æ,n,d,ɹ,u
tone,0,0,0,0,0
stress,-,-,-,-,-
syllabic,+,-,-,-,+
short,-,-,-,-,-
long,-,-,-,-,-
consonantal,-,+,+,-,-
sonorant,+,+,-,+,+
continuant,+,-,-,+,+
delayedRelease,0,0,-,0,0
approximant,+,-,-,+,+


...

,t,aɪ,l,ə,ɹ
tone,0,0,0,0,0
stress,-,-,-,-,-
syllabic,-,+,-,+,-
short,-,-,-,-,-
long,-,-,-,-,-
consonantal,+,-,+,-,-
sonorant,-,+,+,+,+
continuant,-,+,+,+,+
delayedRelease,-,0,0,0,0
approximant,-,+,+,+,+


,z,æ,n,d,ɚ
tone,0,0,0,0,0
stress,-,-,-,-,-
syllabic,-,+,-,-,+
short,-,-,-,-,-
long,-,-,-,-,-
consonantal,+,-,+,+,-
sonorant,-,+,+,-,+
continuant,+,+,-,-,+
delayedRelease,+,0,0,-,0
approximant,-,+,-,-,+


,z,æ,k
tone,0,0,0
stress,-,-,-
syllabic,-,+,-
short,-,-,-
long,-,-,-
consonantal,+,-,+
sonorant,-,+,-
continuant,+,+,-
delayedRelease,+,0,-
approximant,-,+,-


,z,æ,k,ə,ɹ,i
tone,0,0,0,0,0,0
stress,-,-,-,-,-,-
syllabic,-,+,-,+,-,+
short,-,-,-,-,-,-
long,-,-,-,-,-,-
consonantal,+,-,+,-,-,-
sonorant,-,+,-,+,+,+
continuant,+,+,-,+,+,+
delayedRelease,+,0,-,0,0,0
approximant,-,+,-,+,+,+


,z,æ,k
tone,0,0,0
stress,-,-,-
syllabic,-,+,-
short,-,-,-
long,-,-,-
consonantal,+,-,+
sonorant,-,+,-
continuant,+,+,-
delayedRelease,+,0,-
approximant,-,+,-


Finally, we can compare and rank pairs of tokens based on their phonemic similarity:

In [4]:
pd.options.display.max_rows = None
compare(tokens)

ranking pairs by bitwise similarity:   0%|          | 0/703 [00:00<?, ?pair/s]

,a,b,simhash difference (in bits),similarity score
0,(eng) Zach /z æ k/,(eng) Zak /z æ k/,0,1.0
1,(eng) Catherine /k æ θ ə ɹ ə n/,(eng) Catherine /k æ θ ə ɹ ɪ n/,59,0.77
2,(eng) Jenny /d̠ʒ ɛ n i/,(eng) Johnny /d̠ʒ ɑ n i/,65,0.746
3,(eng) Brad /b ɹ æ d/,(eng) Brett /b ɹ ɛ t/,76,0.703
4,(eng) Catherine /k æ θ ə ɹ ə n/,(eng) Nathan /n eɪ θ ə n/,82,0.68
5,(eng) Jennifer /d̠ʒ ɛ n ə f ɚ/,(eng) Jonathan /d̠ʒ ɑ n ə θ ə n/,82,0.68
6,(eng) Carl /k ɑ ɹ l/,(eng) Carlos /k ɑ ɹ l oʊ s/,83,0.676
7,(eng) Catherine /k æ θ ə ɹ ɪ n/,(eng) Nathan /n eɪ θ ə n/,85,0.668
8,(eng) Brad /b ɹ æ d/,(eng) Bradley /b ɹ æ d l i/,86,0.664
9,(eng) Nate /n eɪ t/,(eng) Nick /n ɪ k/,87,0.66
